# Application Context

- Application Context
    - Heart of the application
    - Entry point of the application
    - Encapsulates the BeanFactory (wrapper around the Bean Factory)
    - Provides metadata for Bean creation
    - Allows to create Beans in right order
    - There might be multiple Application Contexts
    - Parent context can interact with child context, not the opposite
- Bean Factory
    - BeanFactory is the IoC container itself
    - Fcilitates the injection of Beans at runtime or startup
    - By default, creates and maintains singleton beans (so same bean is used throughout everywhere)
    - Beans that are not singletons are handled differently
    - Handles all bean injection
    - You need to configure the Bean Factory

# Application Configuration

- Java Configuration
    - Native language syntax
    - Easier IDE integration
    - Compile time checking of configuration
- XML Configuration
    - Legacy code base
    - xml syntax

# Spring Dependency

```
<dependencies>
        <dependency>
            <groupId>org.springframework</groupId>
            <artifactId>spring-core</artifactId>
            <version>${spring.version}</version>
        </dependency>
        <dependency>
            <groupId>org.springframework</groupId>
            <artifactId>spring-context</artifactId>
            <version>${spring.version}</version>
        </dependency>
    </dependencies>
```

# Before configuring the spring

### Application class

```
public class Application {

    public static void main(String[] args) throws Exception {
        GreetingService greetingService = new GreetingService("Hello");
        TimeService timeService = new TimeService(true);
        OutputService outputService = new OutputService(greetingService, timeService);

        for (int i=0;i<5;i++){
            outputService.generateOutput("Frank");
            Thread.sleep(5000);
        }
    }
}
```

### Independent Service : Greeting Service

```
public class GreetingService {

    private final String greeting;

    public GreetingService(String greeting){
        super();
        this.greeting = greeting;
    }

    public String getGreeting(String name){
        return greeting + " " + name;
    }
}
```

### Independent Service : Time Service

```

public class TimeService {

    private static final DateTimeFormatter FORMATTER_24 = DateTimeFormatter.ofPattern("HH:mm:ss");
    private static final DateTimeFormatter FORMATTER_12 = DateTimeFormatter.ofPattern("hh:mm:ss a");

    private final boolean is24;

    public TimeService(boolean is24){
        this.is24 = is24;
    }


    public String getCurrentTime(){
        LocalDateTime now = LocalDateTime.now();
        if (is24) {
            return FORMATTER_24.format(now);
        }
        return FORMATTER_12.format(now);
    }

}

```

### Dependent Service : Output Service

```
public class OutputService {

    private final GreetingService greetingService;
    private final TimeService timeService;

    public OutputService(GreetingService greetingService, TimeService timeService){
        this.greetingService = greetingService;
        this.timeService = timeService;
    }

    public void generateOutput(String name){
        String output = timeService.getCurrentTime() + " " + greetingService.getGreeting(name);
        System.out.println(output);
    }

}
```

# After writing the configuration class

### Configuration class : Registering the service beans

```
@Configuration
public class ApplicationConfig {

    @Autowired
    private GreetingService greetingService;
    @Autowired
    private TimeService timeService;

    @Bean
    public TimeService timeService(){
        return new TimeService(true);
    }
    
    @Bean
    public GreetingService greetingService(){
        return new GreetingService("Hello");
    }

    @Bean
    public OutputService outputService(){
        return new OutputService(greetingService, timeService); // these parameters should be autowired
    }
}
```

### Application class : Using the configuration class

```
public class Application {

    public static void main(String[] args) throws Exception {
        ApplicationContext context = new AnnotationConfigApplicationContext(ApplicationConfig.class);
        OutputService outputService = context.getBean(OutputService.class);

        for (int i=0;i<5;i++){
            outputService.generateOutput("Frank"); // NOT RECOMMENDED : USE CONSTRUCTOR INJECTION
            Thread.sleep(5000);
        }
    }
}
```

# Adding properties and Changing setter injections into consructor injection

### Moving method parameter to constructor parameter

```
public class OutputService {

    private final String name;

    private final GreetingService greetingService;
    private final TimeService timeService;

    public OutputService(GreetingService greetingService, TimeService timeService, String name){ // put parameter here
        this.greetingService = greetingService;
        this.timeService = timeService;
        this.name = name;
    }

    public void generateOutput(){ // name parameter should not be here
        String output = timeService.getCurrentTime() + " " + greetingService.getGreeting(name);
        System.out.println(output);
    }

}
```

### Properties : `application.properties`

```
app.name=Frank
app.greeting=Hello
```

### Configuration class that pulls data from properties

```

@Configuration
@PropertySource("classpath:application.properties") // path to properties file
public class ApplicationConfig {

    @Value("${app.greeting}") // pulling data from properties file
    private String greeting; 
    @Value("${app.name}")
    private String name;

    @Autowired
    private GreetingService greetingService;
    @Autowired
    private TimeService timeService;

    @Bean
    public TimeService timeService(){
        return new TimeService(true);
    }

    @Bean
    public OutputService outputService(){
        return new OutputService(greetingService, timeService, name); // injecting data into service
    }

    @Bean
    public GreetingService greetingService(){
        return new GreetingService(greeting);
    }
}
```

### Application class

```
public class Application {

    public static void main(String[] args) throws Exception {
        ApplicationContext context = new AnnotationConfigApplicationContext(ApplicationConfig.class);
        OutputService outputService = context.getBean(OutputService.class);

        for (int i=0;i<5;i++){
            outputService.generateOutput(); // no more explicit argument for parameter here
            Thread.sleep(5000);
        }
    }
}
```

# Override property variables with environment variables

<center><img src="images/02.01.png"  style="width: 400px, height: 300px;"/></center>
<center><img src="images/02.02.png"  style="width: 400px, height: 300px;"/></center>

# Bean Profile : `ApplicationConfig.java`

```

@Configuration
@PropertySource("classpath:application.properties")
public class ApplicationConfig {

    @Value("${app.greeting}")
    private String greeting;
    @Value("${app.name}")
    private String name;

    @Autowired
    private GreetingService greetingService;
    @Autowired
    private TimeService timeService;

    @Bean
    @Profile("!dev")
    public TimeService timeService(){
        return new TimeService(true);
    }

    @Bean
    @Profile("dev") // another bean configuration for different profile
    public TimeService timeService12(){
        return new TimeService(false);
    }
    

    @Bean
    public GreetingService greetingService(){
        return new GreetingService(greeting);
    }

    @Bean
    public OutputService outputService(){
        return new OutputService(greetingService, timeService, name);
    }

}

```

# Profile in Eclipse : `-Dspring.profiles.active=dev`

<center><img src="images/02.03.png"  style="width: 400px, height: 300px;"/></center>
<center><img src="images/02.04.png"  style="width: 400px, height: 300px;"/></center>
<center><img src="images/02.05.png"  style="width: 400px, height: 300px;"/></center>
<center><img src="images/02.06.png"  style="width: 400px, height: 300px;"/></center>


# SPEL for replacing profile beans

```

@Configuration
@PropertySource("classpath:application.properties")
public class ApplicationConfig {

    @Value("${app.greeting}")
    private String greeting;
    @Value("${app.name}")
    private String name;
    // Setting up the is24 SPEL variable to replace different profile beans
    @Value("#{new Boolean(environment['spring.profiles.active']!='dev')}")
    private boolean is24;

    @Autowired
    private GreetingService greetingService;
    @Autowired
    private TimeService timeService;
    
    
    // One Bean to replace multiple beans of different profiles with SPEL variable is24 
    @Bean
    public TimeService timeService(){
        return new TimeService(is24);
    }
//    @Bean
//    @Profile("!dev")
//    public TimeService timeService(){
//        return new TimeService(true);
//    }
//
//    @Bean
//    @Profile("dev") // another bean configuration for different profile
//    public TimeService timeService12(){
//        return new TimeService(false);
//    }

    @Bean
    public GreetingService greetingService(){
        return new GreetingService(greeting);
    }
    @Bean
    public OutputService outputService(){
        return new OutputService(greetingService, timeService, name);
    }
}
```

# Bean scope

- Singleton
    - default scope
    - one instance of object per context
    - You may declare two beans. But injecting them requires a lot more stuff (NOT RECOMMENDED)
    - Spring keeps the handle of the object
    - be careful with state data. since every class that have the dependency will have access to state data, it may get changed by some other thread.
- Prototype
    - Creates new instance every time it is referenced
    - useful for transient data or types that flex on states
    - Spring hands over that instance and releases its own handle, so when the task is finished, it is automatically disposed off by garbage collector
- Session
    - Works in web environment only
    - One instance of bean per user session
    - Helps to isolate session data from other sessions
    - Spring hands over that instance and releases its own handle, so when the session is finished, it is automatically disposed off by garbage collector
- Request
    - Works in web environment only
    - One instance of bean per user request
    - Spring hands over that instance and releases its own handle, so when the request is finished, it is automatically disposed off by garbage collector


# Proxies

- Proxies add behavior (Aspected behavior added to the classes by the framework)
- in spring everything is proxied
- all classes in spring gets wrapped by at least one proxy
- proxies can only apply their behavior only when they are called by the proxy
- private methods dont get proxy behavior (since they are not exposed to the proxy)
- internal calls (reference method call on same class) dont get proxy behavior (since they are not exposed to the proxy)